# Simulation using NNP

In this tutorial, we provide instructions on running MD using NNP and prior forcefield. To execute the tutorial please install:
* torchmd: https://github.com/torchmd/torchmd
* torchmd-net: https://github.com/torchmd/torchmd-net

For simplicity, we will demonstrate how to run MD using Chignolin. However, we provide the NNPs trained with all targeted proteins as well as muiltiprotein-trained potential. 

In [ ]:
!wget pub.htmd.org/protein_thermodynamics_data/Models.zip
tar -xf Models.zip

# Simulation 

We will use model trained on Chignolin data provided in `Models/cln/model.ckpt` to simulate the coarse-grained system with torchMD. It can be also a network trained in `1-Tutorial-CG-NNP-Training.ipynb`

Forces acting on beads have two components:

1. Prior forces, defined in `prior_force_field.yaml` file. The prior force terms applied here:
* Bonds
* RepulsionCG
* Dihedral
2. Neural Network Potential. The NNP is introduced to torchMD as external force with the use of calculator `torchmdnet.calculators` provided in `torchmd-net` github repository. The external force calculator class has to have `calculate` method that returns a tuple with energy and force tensors.

The input file `simulate.yaml` for the simulation has a series of parameters:

```yaml
device: cuda:0
forcefield: prior_force_field.yaml
forceterms:
- Bonds
- RepulsionCG
- Dihedrals
exclusions: ('bonds')
langevin_gamma: 1
langevin_temperature: 350
log_dir: cln_32trajs_350_ts1
output: output
output_period: 100
precision: double
replicas: 32
rfa: false
save_period: 1000
seed: 1
steps: 5000000
topology: topology/cln.psf
coordinates: starting_xtc/cln_kcenters_32clusters_coords.xtc
temperature: 350
timestep: 1
external:
  module: torchmdnet.calculators
  embeddings: [4, 4, 5, 8, 6, 13, 2, 13, 7, 4]
  file: Models/cln/model.ckpt
```

It can be run as torchMD script:

```bash
python $PATH/torchmd/run.py --conf simulate.yaml
```

where `$PATH` is a path to your `torchmd` repo (not `torchmd-net` linke in `1-Tutorial-CG-NNP-Training.ipynb`)

The simulation GPU and should take a couple of hours on a standard GPU, however can be also run on a CPU.

Now we will go through options in a configuration file:

* First we define `device`, the directory `log_dir` and name of the output files `output`.
* in fields `topology` and `coordinates` we define a file with the topology of simulated system in PSF format and XTC file with initial coordinates for the simulation. In here `starting_xtc/cln_kcenters_32clusters_coords.xtc` contains 32 frames with initial conformations for chignolin, equally distributed over free energy landscape prodiced with all-atom MD. Therefore number of `replicas` is set to 32. 
* `forcefield` defines a file with a forcefield and `forceterms` defines force terms used in the simulation.
* `external` defines parameters for an external force acting on the system, in here, neural network: its location `file`, calculator with appropriate class `module` and `embeddings` that will be used as an input for the network and technically have to correspond to a set of embeddings used in the training. However in case of a general network, transferable across the systems the embeddings do not have to overlap.
* `langevin_gamma` and `langevin_gamma` define parameters of the thermostat
* `temperature` defines the initial temperature of the system, that influences the initial velocities of the atoms.
* `precision` double or single
* `seed` for the calculation
* And finally a set of parameters defining the length of the simulation. `timestep` in fs here is defined as 1fs, however, in case of stable systems the step can be as high as 10fs. `steps` define the length of the simulation, in here it will be 10 000 000 * 1 fs = 10 ns in total. `output_period` indicates that the progress of the simulation and trajectory will be saved every 1 000 steps (1ps) and `save_period` indicates that the output files will be updated every 1 000 steps. 